In [ ]:
import pandas as pd
import pandasgui as pgui
from ratdata import data_manager as dm, plot as rdplot
from peewee import JOIN
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
peaks = pd.read_excel('plots/blinded_power_spectra/peak_detection_merged.xlsx')
key = pd.read_csv('plots/blinded_power_spectra/key.txt', delimiter=':', header=None)
key.columns = ['number', 'filename']
files = dm.RecordingFile.select(dm.RecordingFile.filename, dm.Rat.full_label, dm.Rat.group, dm.RecordingSlice.recording_rejected)\
    .join(dm.Rat).switch(dm.RecordingFile).join(dm.RecordingSlice, join_type=JOIN.LEFT_OUTER)\
        .where((dm.RecordingFile.condition == 'baseline') & ((dm.RecordingSlice.recording_rejected.is_null()) | (dm.RecordingSlice.recording_rejected == False)))
files_rats = pd.DataFrame(list(files.dicts()))

In [ ]:
peaks_files = peaks[['Number', 'Peak location KO', 'Peak low KO', 'Peak high KO']].merge(key, left_on='Number', right_on='number')
peaks_files_rats = pd.merge(peaks_files, files_rats, on='filename', how='left')
peaks_files_rats['peak_width'] = peaks_files_rats['Peak high KO'] - peaks_files_rats['Peak low KO']

In [ ]:
sham_rats = [r.full_label for r in dm.Rat.select(dm.Rat.full_label).where(dm.Rat.group == 'control').order_by(dm.Rat.full_label)]
ohda_rats = [r.full_label for r in dm.Rat.select(dm.Rat.full_label).where(dm.Rat.group == '6OHDA').order_by(dm.Rat.full_label)]

In [ ]:
rdplot.plot_peak_info(peaks_files_rats, 'Peak location KO', ohda_rats, sham_rats[1:], 'plots/blinded_power_spectra/0_peak_locations_baseline', ylim=[12.7, 29.3], figsize=(18, 10))

In [ ]:
rdplot.plot_peak_info(peaks_files_rats, 'peak_width', ohda_rats, sham_rats[1:], 'plots/blinded_power_spectra/0_peak_widths_baseline', ylim=[0.7, 8.3], figsize=(18, 10))